In [1]:
# Copyright 2019 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Cascade (HD-CNN Model Deriative)

## Objective

This notebook demonstrates building a hierachical image classifer based on a HD-CNN deriative which uses cascading classifers to predict the class of a label from a coarse to finer classes. 

In this demonstration, we have two classes in the heirarchy: fruits and varieties of fruit. The model will first predict the coarse class (type of fruit) and then within that class of fruit, the variety. For example, if given an image of Apple Granny Smith, it would first predict 'Apple' (fruit) and then predict the 'Apple Granny Smith'.

This deriative of the HD-CNN is designed to demonstrate both the methodology of heirarchical classification, as well as design improvements not available at the time (2014) when the model was first published [Zhicheng Yan](https://arxiv.org/abs/1410.0736).

### General Approach

Our HD-CNN deriative archirecture consists of:

    1. An stem convolutional block.
        - The output from the stem convolutional head is shared with the coarse and finer classifiers 
        (referred to as the shared layers in the paper).
    2. A coarse classifier.
        - A Convolution and Dense layers for classifying the coarse level class. 
    3. A set of finer classifiers, one per coarse level class.
        - A Convolution and Dense layers per coarse level class for classifying the corresponding finer 
        level class.
    4. A conditional execution step for predicting a specific finer classifier based on the output of the 
       coarse classifier.
        - The coarse level classifier is predicted.
        - The index of the prediction is used to select a finer classifier.
        - An im-memory copy of the shared bottleneck layer (i.e., last convolution layer in stem) is passed as the
          input to the finer level classifier.
    

Our HD-CNN deriative is trained as follows:

    1. Train the coarse level classifier using the coarse level labels in the dataset.
    
<img src='arch-1.png'>
    
    2. Train the finer level classifier per coarse level class, using the corresponding subset (with finer
       labels) from the dataset.
       
<img src='arch-2.png'>

<br/>      
## Dataset

We will be using the Fruits-360 dataset, which was formerly a Kaggle competition. It consists of images of fruit labeled by fruit type and the variety. 

    1. There are a total of 47 types of fruit (e.g., Apple, Orange, Pear, etc) and 81 varieties.
    2. On average, there are 656 images per variety.
    3. Each image is 128x128 RGB.
    
<div>
<img src='Training/Apple/Apple Golden 2/0_100.jpg' style='float: left'>
<img src='Training/Apple/Apple Red 1/0_100.jpg'  style='float: left'>
<img src='Training/Apple/Apple Red 1/0_100.jpg' style='float: left'>
<img src='Training/Orange/Orange/0_100.jpg' style = 'float: left'>
<img src='Training/Pear/Pear/0_100.jpg' style = 'float: left'>
</div>


## Objective

The objective is to train a hierarchical image classifier (coarse and then finer label) using a cascading layer architecture. First, the shared layers and coarse classifier are trained. Then the cascading finer classifiers are trained. 

For prediction, the outcome (softmax) of the coarse classifier will conditionally execute the corresponding finer classifier and reuse the feature maps from the shared layers.

## Costs

This notebook requires 17GB of memory. It will not run on a Standard TF JaaS instance (15GB). You will need to select an instance with memory > 17GB.

## Prerequisites

Download the Fruits 360 dataset from GCS public bucket into this JaaS instance.

Some of the cells in the notebook display images. The images will not appear until the cell for copying the training data/misc from GCS into the JaaS instance is executed.

In [1]:
!gsutil cp gs://cloud-samples-data/air/fruits360/fruits360-combined.zip .
!ls
!unzip -qn fruits360-combined.zip

Copying gs://cloud-samples-data/air/fruits360/fruits360-combined.zip...
==> NOTE: You are downloading one or more large file(s), which would            
run significantly faster if you enabled sliced object downloads. This
feature is enabled by default but requires that compiled crcmod be
installed (see "gsutil help crcmod").

/ [1 files][230.9 MiB/230.9 MiB]                                                
Operation completed over 1 objects/230.9 MiB.                                    
arch-1.png		model-coarse.h5    model-finer-1.h5  model-finer-8.h5
arch-2.png		model-finer-0.h5   model-finer-2.h5  model-finer-9.h5
cascade-gap.ipynb	model-finer-10.h5  model-finer-3.h5  model.h5
cascade.ipynb		model-finer-11.h5  model-finer-4.h5  Save
fruits360-Cascade.zip	model-finer-12.h5  model-finer-5.h5  Training
fruits360-combined.zip	model-finer-13.h5  model-finer-6.h5
fruits.h5.h5		model-finer-14.h5  model-finer-7.h5


## Getting Started

We will be using the fully frameworks and Python modules:

    1. Keras framework for building and training models.
    2. Keras builtin models (resnet50).
    3. Keras preprocessing for feeding and augmenting the dataset during training.
    4. Gap data engineering framework for preprocessing the image data.
    5. Numpy for general image/matrix manipulation.


In [2]:
import os
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import GlobalAveragePooling2D, Dense
from keras import Sequential, Model, Input
from keras.layers import Conv2D, Flatten, MaxPooling2D, Dense, Dropout, BatchNormalization, ReLU
from keras import Model, optimizers
from keras.models import load_model
from keras.utils import to_categorical
import keras.layers as layers
from sklearn.model_selection import train_test_split
import tensorflow as tf
import numpy as np
import cv2

Using TensorFlow backend.


## Make Datasets

### Make Coarse Category Dataset

This makes the by fruit type dataset.

In [3]:
def Fruits(root):
    n_label = 0
    images = []
    labels = []
    classes = {}
    
    os.chdir(root)
    classes_ = os.scandir('./')
    for class_ in classes_:
        print(class_.name)
        os.chdir(class_.name)
        classes[class_.name] = n_label

        # Finer Level Subdirectories per Coarse Level
        subclasses = os.scandir('./')
        for subclass in subclasses:
            os.chdir(subclass.name)
            files = os.listdir('./')
            for file in files:
                image = cv2.imread(file)
                images.append(image)
                labels.append(n_label)
                
            os.chdir('../')

        os.chdir('../')
        n_label += 1
    os.chdir('../')
    images = np.asarray(images)
    images = (images / 255.0).astype(np.float32)
    labels = to_categorical(labels, n_label)
    print("Images", images.shape, "Labels", labels.shape, "Classes", classes)

    # Split the processed image dataset into training and test data
    x_train, x_test, y_train, y_test = train_test_split(images, labels, test_size=0.20, shuffle=True)
    return x_train, x_test, y_train, y_test, classes

### Make Finer Category Datasets

This makes the by Fruit Variety datasets

In [4]:
def Varieties(root):
    ''' Generate Cascade (Finer) Level Dataset for Fruit Varieties'''

    datasets = {}
    
    os.chdir(root)
    fruits = os.scandir('./')
    for fruit in fruits:
        n_label = 0
        images = []
        labels = []
        classes = {}
        print('FRUIT', fruit.name)
        os.chdir(fruit.name)
        varieties = os.scandir('./')
        for variety in varieties:
            print('VARIETY', variety.name)
            classes[variety.name] = n_label
            os.chdir(variety.name)
            files = os.listdir('./')
            for file in files:
                image = cv2.imread(file)
                images.append(image)
                labels.append(n_label)
            os.chdir('../')
            n_label += 1
        images = np.asarray(images)
        images = (images / 255.0).astype(np.float32)
        labels = to_categorical(labels, n_label)
        x_train, x_test, y_train, y_test = train_test_split(images, labels, test_size=0.20, shuffle=True)
        datasets[fruit.name] = (x_train, x_test, y_train, y_test, classes)
        os.chdir('../')
        print("IMAGES", x_train.shape, y_train.shape, "CLASSES", classes)
    os.chdir('../')
    return datasets

### Generate the preprocessed Coarse Dataset

In [5]:
!free -m
x_train, x_test, y_train, y_test, fruits_classes = Fruits('Training')
!free -m

              total        used        free      shared  buff/cache   available
Mem:          64146        8888       45049         101       10208       54503
Swap:         65187        5628       59559
Avocado
Rambutan
Pineapple
Kiwi
Cantaloupe
Apple
Tamarillo
Physalis
Plum
Pitahaya
Guava
Limes
Grapefruit
Peach
Pomegranate
Nectarine
Apricot
Banana
Cherry
Mulberry
Raspberry
Cactus Fruit
Grape
Mandarine
Granadilla
Carambula
Passion Fruit
Lychee
Quince
Maracuja
Strawberry
Tangelo
Huckleberry
Orange
Dates
Melon
Pepino
Clementine
Papaya
Mango
Tomato
Salak
Kaki
Pear
Cocos
Lemon
Kumquats
Images (51258, 100, 100, 3) Labels (51258, 47) Classes {'Avocado': 0, 'Rambutan': 1, 'Pineapple': 2, 'Kiwi': 3, 'Cantaloupe': 4, 'Apple': 5, 'Tamarillo': 6, 'Physalis': 7, 'Plum': 8, 'Pitahaya': 9, 'Guava': 10, 'Limes': 11, 'Grapefruit': 12, 'Peach': 13, 'Pomegranate': 14, 'Nectarine': 15, 'Apricot': 16, 'Banana': 17, 'Cherry': 18, 'Mulberry': 19, 'Raspberry': 20, 'Cactus Fruit': 21, 'Grape': 22, 'Mandarine

### Split Coarse Dataset (by Fruit) into Train, Validation and Test

First split into train and test. Then split out 10% of train to use for validation during training.

    - Train: 80%
        - Train: 90%
        - Validation: 10%
    - Test : 20%

In [6]:
# Split out 10% of Train to use for Validation
pivot = int(len(x_train) * 0.9)
x_val = x_train[pivot:]
y_val = y_train[pivot:]
x_train = x_train[:pivot]
y_train = y_train[:pivot]

print("train", x_train.shape, y_train.shape)
print("val  ", x_val.shape, y_val.shape)
print("test ", x_test.shape, y_test.shape)
!free -m

train (36905, 100, 100, 3) (36905, 47)
val   (4101, 100, 100, 3) (4101, 47)
test  (10252, 100, 100, 3) (10252, 47)
              total        used        free      shared  buff/cache   available
Mem:          64146       16255       37683         101       10207       47136
Swap:         65187        5628       59559


## Make Trainers

Create the routines we will use for training.

### Make Feeder

Prepare the Feeder mechanism for training the neural networkm using ImageDataGenerator. 

Add image augmentation for:

    1. Horizontal Flip
    2. Verticial  Flip
    3. Random Rotation +/- 30 degrees

In [7]:
def Feeder():
    datagen = ImageDataGenerator(horizontal_flip=True, vertical_flip=True, rotation_range=30)
    return datagen

### Make Trainer

Prepare a training session:

    1. Epochs defaults to 10
    2. Batch size defaults to 32
    3. Train with validation data
    4. Final evaluation with test data (holdout set).

In [8]:
def Train(model, datagen, x_train, y_train, x_test, y_test, epochs=10, batch_size=32):
    model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size, shuffle=True),
                    steps_per_epoch=len(x_train) / batch_size, epochs=epochs, verbose=1, validation_data=(x_test, y_test))
    scores = model.evaluate(x_train, y_train, verbose=1)
    print("Train", scores)

## Make Model

### Stem Convolutional Block (Base Model)

We will use this base model as the stem convolutional block of cascading model:
    1. The output of this model are a set of pooled feature maps.
    2. The last layer that produces this set of pooled feature maps is referred to as the bottleneck layer.

### Coarse Classifier

The coarse classifier is an independent block layer for classifying the coarse level label:

    1. Input is the bottleneck layer from the stem convolutional block.
    2. Layer consists of a convolution layer and a dense layer, where the dense layer is the classifier.
    
### Finer Classifier

The finer classifiers are a set of independent block layers for classifying the finer label. There is one finer classifier per unique coarse level label.

    1. Input is the bottleneck layer from the stem convolutional block.
    2. Layer consists of a convolution layer and a dense layer, where the dense layer is the classifier.
    3. The finer classifer is conditionally executed based on the softmax output from the coarse classifier.


### ResNet for Transfer Learning

Use a prebuilt Keras model (ResNet 50). Either as:

    1. Transfer Learning: The layers are pretrained with imagenet weights.
    2. Full Training: layers are not pretrained (weights = None)

In [9]:
def ResNet(shape=(128, 128, 3), nclasses=47, optimizer='adam', weights=None):
    base_model = ResNet50(weights=weights, include_top=False, input_shape=shape)

    for i, layer in enumerate(base_model.layers):
        # first: train only the top layers (which were randomly initialized) for Transfer Learning
        if weights is not None:
            layer.trainable = False
       
    # label the last convolutional layer in the base model as the bottleneck
    layer.name = 'bottleneck'
    
    # Get the last convolutional layer of the ResNet base model
    x = base_model.output
    
    # add a global spatial average pooling layer
    x = GlobalAveragePooling2D()(x)
    # let's add a fully-connected layer
    #x = Dense(1024, activation='relu')(x)
    # and a logistic layer 
    predictions = Dense(nclasses, activation='softmax')(x)
    # this is the model we will train
    model = Model(inputs=base_model.input, outputs=predictions)

    # compile the model (should be done *after* setting layers to non-trainable)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    model.summary()
    return model

### Simple ConvNet

The stem convolutional block consists of a mini-VGG, which consists of:

    1. A convolutional input (stem)
    2. Three convolutional groups, each doubling the number of filers.
    3. Each convolutional group consists of one convolutional block.
    4. A dropout of 50% is added to the first convolutional group.
    
The coarse classifier consists of:

    1. A 1024 none dense layer
    2. A 47 node dense layer for classification.

In [10]:
def ConvNet(shape=(128, 128, 3), nclasses=47, optimizer='adam'):
    model = Sequential()
    # stem convolutional group
    model.add(Conv2D(16, (3,3), padding='same', activation='relu', input_shape=shape))

    # conv block - double filters
    model.add(Conv2D(32, (3,3), padding='same'))
    model.add(ReLU())    
    model.add(Dropout(0.50)) 
    model.add(MaxPooling2D((2,2)))
    
    # conv block - double filters
    model.add(Conv2D(64, (3,3), padding='same'))
    model.add(ReLU())
    model.add(MaxPooling2D((2,2)))
    
    # conv block - double filters + bottleneck layer
    model.add(Conv2D(128, (3,3), padding='same', activation='relu'))
    model.add(MaxPooling2D((2,2), name="bottleneck"))
    
    # dense block
    model.add(Flatten())
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(0.25))
    
    # classifier
    model.add(Dense(nclasses, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    model.summary()
    return model

## Start Training

    1. Train the Coarse Classifier
    2. Add Finer Classifiers
    4. Train the Finer Classifiers

### Generate Coarse Model

Choose between:

    1. A untrained simple VGG CovNet as Stem Convolution Group, or
    2. Pre-trained ResNet50 (imagenet weights) for Transfer Learning

In [11]:
# Select the model for the stem convolutional group (shared layers)
stem = 'ConvNet'
if stem == 'ConvNet':
    model = ConvNet(shape=(100, 100, 3))
elif stem == 'ResNet-imagenet':
    model = ResNet(weights='imagenet', optimizer='adagrad')
elif stem == 'ResNet':
    model = ResNet()
# load previously stored model
else:
    model = load_model('model.h5')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 100, 100, 16)      448       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 100, 100, 32)      4640      
_________________________________________________________________
re_lu_1 (ReLU)               (None, 100, 100, 32)      0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 100, 100, 32)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 50, 50, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)    

### Train the Coarse Model

In [12]:
datagen = Feeder()
Train(model, datagen, x_train, y_train, x_val, y_val, 5)

scores = model.evaluate(x_test, y_test, verbose=1)
print("Test", scores)

Instructions for updating:
Use tf.cast instead.
Epoch 1/5
1154/1153 [==============================] - 492s 427ms/step - loss: 0.7688 - acc: 0.7665 - val_loss: 0.4038 - val_acc: 0.8800
Epoch 2/5
1154/1153 [==============================] - 489s 424ms/step - loss: 0.1687 - acc: 0.9431 - val_loss: 0.1588 - val_acc: 0.9537
Epoch 3/5
1154/1153 [==============================] - 487s 422ms/step - loss: 0.1329 - acc: 0.9529 - val_loss: 0.1037 - val_acc: 0.9659
Epoch 4/5
1154/1153 [==============================] - 489s 424ms/step - loss: 0.1239 - acc: 0.9589 - val_loss: 0.2824 - val_acc: 0.9056
Epoch 5/5
36905/36905 [==============================] - 90s 2ms/step
Train [0.1527849808228305, 0.9426364991209756]
10252/10252 [==============================] - 26s 3ms/step
Test [0.17052910335498586, 0.935914943425673]


### Save the Coarse Model


In [13]:
# Save the model and weights
model.save("model-coarse.h5")

### Prepare Coarse CNN for cascade training

    1. Freeze all layers
    2. Find bottleneck layer

In [14]:
def Bottleneck(model):
    for layer in model.layers:
        layer.trainable = False
        if layer.name == 'bottleneck':
            bottleneck = layer

    print("BOTTLENECK", bottleneck.output.shape)
    return bottleneck

### Generate the preprocessed Finer Datasets

### Split Finer (by Variety) Datasets into Train, Validation and Test

    1. For each fruit type, split the corresponding variety images into train, validation and test.
    2. Save each split dataset in a dictionary, using the fruit name as the key.

In [15]:
# Converse memory by releasing training data for coarse model
import gc
x_train = y_train = x_val = y_val = x_test = y_test = None
gc.collect()

5

In [16]:
varieties_datasets = Varieties('Training')
for key, dataset in varieties_datasets.items():
    
    _x_train, _x_test, _y_train, _y_test, classes = dataset
            
    # Separate out 10% of train for validation
    pivot = int(len(_x_train) * 0.9)
    _x_val = _x_train[pivot:]
    _y_val = _y_train[pivot:]
    _x_train = _x_train[:pivot]
    _y_train = _y_train[:pivot]
    
    # save the dataset for this fruit (key)
    varieties_datasets[key] = { 'classes': classes, 'train': (_x_train, _y_train), 'val': (_x_val, _y_val), 'test': (_x_test, _y_test) }
    
!free -m

FRUIT Avocado
VARIETY Avocado
VARIETY Avocado ripe
IMAGES (981, 100, 100, 3) (981, 2) CLASSES {'Avocado': 0, 'Avocado ripe': 1}
FRUIT Rambutan
VARIETY Rambutan
IMAGES (524, 100, 100, 3) (524, 1) CLASSES {'Rambutan': 0}
FRUIT Pineapple
VARIETY Pineapple
VARIETY Pineapple Mini
IMAGES (1048, 100, 100, 3) (1048, 2) CLASSES {'Pineapple': 0, 'Pineapple Mini': 1}
FRUIT Kiwi
VARIETY Kiwi
IMAGES (497, 100, 100, 3) (497, 1) CLASSES {'Kiwi': 0}
FRUIT Cantaloupe
VARIETY Cantaloupe 1
VARIETY Cantaloupe 2
IMAGES (1049, 100, 100, 3) (1049, 2) CLASSES {'Cantaloupe 1': 0, 'Cantaloupe 2': 1}
FRUIT Apple
VARIETY Apple Golden 2
VARIETY Apple Red 3
VARIETY Apple Red Yellow
VARIETY Apple Granny Smith
VARIETY Apple Red 2
VARIETY Apple Braeburn
VARIETY Apple Golden 3
VARIETY Apple Red Delicious
VARIETY Apple Red 1
VARIETY Apple Golden 1
IMAGES (5170, 100, 100, 3) (5170, 10) CLASSES {'Apple Golden 2': 0, 'Apple Red 3': 1, 'Apple Red Yellow': 2, 'Apple Granny Smith': 3, 'Apple Red 2': 4, 'Apple Braeburn': 5, 'A

### Add Each Cascade (Finer) Classifier

    1. Get the bottleneck layer for the coarse CNN
    2. Add an independent finer classifier per fruit from the bottleneck layer

In [17]:
bottleneck = Bottleneck(model)
cascades = []
for key, val in varieties_datasets.items():
    classes = val['classes']
    print("KEY", key, classes)
    # if only one subclassifier, then skip (i.e., coarse == finer)
    if len(classes) == 1:
        continue
    x = layers.Conv2D(128, (3,3), padding='same', activation='relu')(bottleneck.output)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2,2))(x)
    
    x = layers.Flatten()(bottleneck.output)
    x = layers.Dense(1024, activation='relu')(x)
    x = layers.Dense(len(classes), activation='softmax', name=key.replace(' ', ''))(x)
    cascades.append(x)

BOTTLENECK (?, 12, 12, 128)
KEY Avocado {'Avocado': 0, 'Avocado ripe': 1}
KEY Rambutan {'Rambutan': 0}
KEY Pineapple {'Pineapple': 0, 'Pineapple Mini': 1}
KEY Kiwi {'Kiwi': 0}
KEY Cantaloupe {'Cantaloupe 1': 0, 'Cantaloupe 2': 1}
KEY Apple {'Apple Golden 2': 0, 'Apple Red 3': 1, 'Apple Red Yellow': 2, 'Apple Granny Smith': 3, 'Apple Red 2': 4, 'Apple Braeburn': 5, 'Apple Golden 3': 6, 'Apple Red Delicious': 7, 'Apple Red 1': 8, 'Apple Golden 1': 9}
KEY Tamarillo {'Tamarillo': 0}
KEY Physalis {'Physalis': 0, 'Physalis with Husk': 1}
KEY Plum {'Plum': 0}
KEY Pitahaya {'Pitahaya Red': 0}
KEY Guava {'Guava': 0}
KEY Limes {'Limes': 0}
KEY Grapefruit {'Grapefruit White': 0, 'Grapefruit Pink': 1}
KEY Peach {'Peach': 0, 'Peach Flat': 1}
KEY Pomegranate {'Pomegranate': 0}
KEY Nectarine {'Nectarine': 0}
KEY Apricot {'Apricot': 0}
KEY Banana {'Banana': 0, 'Banana Red': 1}
KEY Cherry {'Cherry Rainier': 0, 'Cherry 2': 1, 'Carambula': 2, 'Cherry Wax Yellow': 3, 'Cherry Wax Black': 4, 'Cherry 1': 5, 

### Compile each finer classifier

In [18]:
classifiers = []
for cascade in cascades:
    _model = Model(model.input, cascade)
    _model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    _model.summary()
    classifiers.append(_model)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1_input (InputLayer)  (None, 100, 100, 3)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 100, 100, 16)      448       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 100, 100, 32)      4640      
_________________________________________________________________
re_lu_1 (ReLU)               (None, 100, 100, 32)      0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 100, 100, 32)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 50, 50, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 50, 50, 64)        18496     
__________

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1_input (InputLayer)  (None, 100, 100, 3)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 100, 100, 16)      448       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 100, 100, 32)      4640      
_________________________________________________________________
re_lu_1 (ReLU)               (None, 100, 100, 32)      0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 100, 100, 32)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 50, 50, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 50, 50, 64)        18496     
__________

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1_input (InputLayer)  (None, 100, 100, 3)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 100, 100, 16)      448       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 100, 100, 32)      4640      
_________________________________________________________________
re_lu_1 (ReLU)               (None, 100, 100, 32)      0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 100, 100, 32)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 50, 50, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 50, 50, 64)        18496     
__________

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1_input (InputLayer)  (None, 100, 100, 3)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 100, 100, 16)      448       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 100, 100, 32)      4640      
_________________________________________________________________
re_lu_1 (ReLU)               (None, 100, 100, 32)      0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 100, 100, 32)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 50, 50, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 50, 50, 64)        18496     
__________

### Train the finer classifiers

In [19]:
for classifier in classifiers:
    # get the output layer for this subclassifier
    last = classifier.layers[len(classifier.layers)-1]
    print(last, last.name)
    
    # find the corresponding variety dataset
    for key, dataset in varieties_datasets.items():
        if key == last.name:
            x_train, y_train = dataset['train']
            x_val, y_val     = dataset['val']

            datagen = Feeder()
            Train(classifier, datagen, x_train, y_train, x_val, y_val, 5)


<keras.layers.core.Dense object at 0x7f622995a9b0> Avocado
Epoch 1/5
28/27 [==============================] - 8s 274ms/step - loss: 0.0588 - acc: 0.9620 - val_loss: 1.1626e-04 - val_acc: 1.0000
Epoch 2/5
28/27 [==============================] - 5s 196ms/step - loss: 3.6445e-06 - acc: 1.0000 - val_loss: 4.3129e-05 - val_acc: 1.0000
Epoch 3/5
28/27 [==============================] - 5s 194ms/step - loss: 8.5374e-07 - acc: 1.0000 - val_loss: 4.0188e-05 - val_acc: 1.0000
Epoch 4/5
28/27 [==============================] - 5s 190ms/step - loss: 8.1645e-07 - acc: 1.0000 - val_loss: 3.9653e-05 - val_acc: 1.0000
Epoch 5/5
882/882 [==============================] - 2s 2ms/step
Train [3.1851555930955815e-05, 1.0]
<keras.layers.core.Dense object at 0x7f62297766a0> Pineapple
Epoch 1/5
30/29 [==============================] - 6s 216ms/step - loss: 0.0656 - acc: 0.9666 - val_loss: 8.1402e-04 - val_acc: 1.0000
Epoch 2/5
30/29 [==============================] - 5s 181ms/step - loss: 1.3917e-05 - acc: 1

### Evaluate the Model

    1. Evaluate the Model for each finer classifier.

In [20]:
for classifier in classifiers:
    # get the output layer for this subclassifier
    last = classifier.layers[len(classifier.layers)-1]
    print(last, last.name)
    
    # find the corresponding variety dataset
    for key, dataset in varieties_datasets.items():
        if key == last.name:
            x_test, y_test = dataset['test']
            scores = classifier.evaluate(x_test, y_test, verbose=1)
            print("Test", scores)



<keras.layers.core.Dense object at 0x7f622995a9b0> Avocado
246/246 [==============================] - 1s 3ms/step
Test [3.334964506992525e-05, 1.0]
<keras.layers.core.Dense object at 0x7f62297766a0> Pineapple
263/263 [==============================] - 1s 2ms/step
Test [0.0016150613485378684, 1.0]
<keras.layers.core.Dense object at 0x7f62296131d0> Cantaloupe
263/263 [==============================] - 1s 3ms/step
Test [1.6362951567596597e-07, 1.0]
<keras.layers.core.Dense object at 0x7f6229493ac8> Apple
1293/1293 [==============================] - 3s 2ms/step
Test [0.5182560322848015, 0.7973704563492688]
<keras.layers.core.Dense object at 0x7f62292ad828> Physalis
263/263 [==============================] - 1s 2ms/step
Test [2.5120300804652986e-06, 1.0]
<keras.layers.core.Dense object at 0x7f60ae144358> Grapefruit
263/263 [==============================] - 1s 2ms/step
Test [2.3878306024448906e-05, 1.0]
<keras.layers.core.Dense object at 0x7f60adfcbfd0> Peach
263/263 [======================

### Save the Finer Models

In [21]:
n = 0
for classifier in classifiers:
    classifier.save('model-finer-' + str(n) + '.h5')
    n += 1

## Let's do some cascading predictions

We will take one random selected image per type of fruit, and:

    1. Run the image through the coarse classifier (by fruit).
    2. Based on the predicted output, select the corresponding finer classifier (by variety).
    3. Run the image through the corresponding finer classifier.

In [22]:
import random

# Let's make a prediction for each type of fruit
for key, dataset in varieties_datasets.items():
    
    # Get the variety test data for this type of fruit
    x_test, y_test = dataset['test']
    
    # pick a random image in the variety datast
    index = random.randint(0, len(x_test))
    
    # use the coarse model to predict the type of fruit
    yhat = np.argmax( model.predict(x_test[index:index+1]) )
    
    # let's find the class name (type of fruit) for this predicted label
    for fruit, label in fruits_classes.items():
        if label == yhat:
            break
    
    print("Yhat", yhat, "Coarse Prediction", key, "=", fruit)
    
    # Prediction was correct
    if key == fruit:
        if len(dataset['classes']) == 1:
            print("No Finer Classifier")
            continue
            
        # find the corresponding finer classifier for this type of fruit
        for classifier in classifiers:
            # get the output layer for this subclassifier
            last = classifier.layers[len(classifier.layers)-1]
            if last.name == fruit:
                # use the finer model to predict the variety of this type of fruit
                yhat = np.argmax(classifier.predict(x_test[index:index+1]))
                for variety, value in dataset['classes'].items():
                    if value == np.argmax(y_test[index]):
                        break
                for yhat_variety, value in dataset['classes'].items():
                    if value == yhat:
                        break
                print("Yhat", yhat, "Finer  Prediction", variety, "=", yhat_variety)
                break

Yhat 0 Coarse Prediction Avocado = Avocado
Yhat 1 Finer  Prediction Avocado ripe = Avocado ripe
Yhat 20 Coarse Prediction Rambutan = Raspberry
Yhat 2 Coarse Prediction Pineapple = Pineapple
Yhat 0 Finer  Prediction Pineapple = Pineapple
Yhat 3 Coarse Prediction Kiwi = Kiwi
No Finer Classifier
Yhat 4 Coarse Prediction Cantaloupe = Cantaloupe
Yhat 1 Finer  Prediction Cantaloupe 2 = Cantaloupe 2
Yhat 5 Coarse Prediction Apple = Apple
Yhat 4 Finer  Prediction Apple Red 2 = Apple Red 2
Yhat 6 Coarse Prediction Tamarillo = Tamarillo
No Finer Classifier
Yhat 7 Coarse Prediction Physalis = Physalis
Yhat 1 Finer  Prediction Physalis with Husk = Physalis with Husk
Yhat 8 Coarse Prediction Plum = Plum
No Finer Classifier
Yhat 9 Coarse Prediction Pitahaya = Pitahaya
No Finer Classifier
Yhat 10 Coarse Prediction Guava = Guava
No Finer Classifier
Yhat 11 Coarse Prediction Limes = Limes
No Finer Classifier
Yhat 12 Coarse Prediction Grapefruit = Grapefruit
Yhat 1 Finer  Prediction Grapefruit Pink = Gr

### End of Notebook

In [ ]:
#     extractfeatures = Model(input=model.input, output=model.get_layer('bottleneck').output)